In [234]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import requests, re, json ,pprint
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import urllib.request
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
from urllib.parse import urlparse
from time import sleep
import datetime
import pytz # new import
new_timezone = pytz.timezone("US/Eastern")
import spacy
import en_core_web_sm
import en_core_web_lg
import en_vectors_web_lg
import en_core_web_md
import csv
from dateparser.search import search_dates
import arrow
import datetime
from selenium.webdriver.support import expected_conditions as EC


In [270]:
driver_path='./chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')   
driver = webdriver.Chrome(driver_path,options=options)
# driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

In [112]:
def reload_test_site():
    my_url = 'https://www.youtube.com/watch?v=A1tqTQkC36A&amp;lc=Ugw0nq07e-G5JyUsl094AaABAg'
    driver.get(my_url)
    sleep(1)

In [116]:
def get_driver(driver,my_url):
#     my_url = 'https://www.youtube.com/watch?v=A1tqTQkC36A&amp;lc=Ugw0nq07e-G5JyUsl094AaABAg'
    driver.get(my_url)
    return driver
#     sleep(1)

## Define User ID

In [327]:
def extract_transcript(driver):
    sleep(2)
    driver.implicitly_wait(10)
    test = driver.find_element_by_xpath("//button[@aria-label='More actions']");
    test.click()
    driver.implicitly_wait(10)
    test_out = driver.find_element_by_xpath('//yt-formatted-string[text() ="Open transcript"]')
    test_out.click()
    driver.implicitly_wait(10)
    test_out = driver.find_element_by_xpath('//div[@id="body"]/ytd-transcript-body-renderer')
    test_body = test_out.text.encode('utf-8')    
    
    return test_body.decode('utf-8','ignore')

## Extract from List of Videos

In [322]:
csv_input = 'batch2_klavan2.csv'
csv_output = 'transcript_' + csv_input
filenames = pd.read_csv(csv_input)

In [323]:
filenames.video_url.head()
vid_id_df =filenames.video_url.unique()


In [ ]:
with open(csv_output, 'w', newline='') as csvfile:
    fieldnames = ['url', 'transcript']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,
            delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for urls in vid_id_df:
        try:
            driver.get(urls)
            text = extract_transcript(driver)
            x = {"url":urls,"transcript":text}
            writer.writerow(x)
        except:
            pass
